In [1]:
from pathlib import Path
import pathlib

import base64


# Assuming the images are now in the 'image_assets' folder and the path is correct
image_files = list(Path("ideas/image_assets").glob("*"))

# Read all images into base64 strings
base64_images = {}
for image_path in image_files:
    with open(image_path, "rb") as image_file:
        base64_img_string_raw = base64.b64encode(image_file.read()).decode("utf-8")
        base64_images[
            image_path.name
        ] = f"data:image/png;base64,{base64_img_string_raw}"


In [2]:
import anywidget
from traitlets import Unicode, Int, Tuple, observe, Bool, List, Any, Dict
from pathlib import Path
import pathlib
import base64

class TldrawSetImage(anywidget.AnyWidget):
    @staticmethod
    def base64_to_image_dimensions(base64_img_string):
        base64_img_string_only = base64_img_string.split(",")[1]
        decoded_bytes = base64.b64decode(base64_img_string_only)
        if decoded_bytes[:8] != b"\x89PNG\r\n\x1a\n":
            raise ValueError("Invalid PNG file")
        ihdr_start = 8
        ihdr_end = decoded_bytes.find(b"IHDR") + 4 + 8
        ihdr_chunk = decoded_bytes[ihdr_start:ihdr_end]
        image_width = int.from_bytes(ihdr_chunk[8:12], byteorder="big")
        image_height = int.from_bytes(ihdr_chunk[12:16], byteorder="big")
        return image_width, image_height

    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    def set_image(self, base64img):
        if not base64img:
            raise ValueError("No image provided")

        image_width, image_height = self.base64_to_image_dimensions(base64img)
        self.image_dimensions = (int(image_width / 2), int(image_height / 2))
        self.base64img = base64img

    base64img = Unicode("").tag(sync=True)
    image_dimensions = Tuple(Int(), Int(), default_value=(0, 0)).tag(sync=True)

    
    length = Int(100).tag(sync=True)
    coord = List().tag(sync=True)

    path_root = pathlib.Path.cwd()
    _esm = path_root / "src" / "tldraw" / "static" / "image_set.js"
    _css = path_root / "src" / "tldraw" / "static" / "image_set.css"

In [3]:
widget = TldrawSetImage()
widget

TldrawSetImage(image_dimensions=(0, 0))

In [4]:
widget.set_image(base64img = base64_images["im3.png"])

In [5]:
widget.set_image(base64img = base64_images["im2.png"])

In [6]:
widget.coord

[]